## 1 Setup Notebook and Load Variables

In [ ]:
import json
import os
from time import sleep
from tango import AttributeProxy, DeviceProxy
from datetime import datetime, timedelta, timezone
from astropy.time import Time

In [ ]:
TANGO_HOST = "10.164.10.219:10000"
RECEPTORS = ["SKA001", "SKA036"]
SKA_EPOCH = "1999-12-31T23:59:28Z"

# Config file locations
DATA_DIR = "../../data/mid_telescope"
TMC_DIR = f"{DATA_DIR}/tmc"
INITIAL_PARAM = f"{DATA_DIR}/cbf/sys_params/initial_system_param.json"
CSP_ASSIGN_RESOURCES = f"{DATA_DIR}/csp/assign_resources.json"
TMC_ASSIGN_RESOURCES = f"{TMC_DIR}/assign_resources.json"
CONFIGURE_SCAN = f"{TMC_DIR}/configure_scan.json"
DELAY_MODEL = f"{TMC_DIR}/delay_model.json"
SCAN_JSON = f"{TMC_DIR}/scan.json"

# Device FQDNs
csp_controller = "mid-csp/control/0"
csp_server = "mid-csp/subarray/01"
sdp_subarray = "mid-sdp/subarray/01"
csp_subarray_leaf_node = "ska_mid/tm_leaf_node/csp_subarray01"
sdp_subarray_leaf_node = "ska_mid/tm_leaf_node/sdp_subarray01"
csp_master_leaf_node = "ska_mid/tm_leaf_node/csp_master"
tm_server = "ska_mid/tm_leaf_node/csp_subarray_01"

SIMULATED_DELAY = False

In [ ]:
os.environ["TANGO_HOST"] = TANGO_HOST

csp_controller_dp = DeviceProxy(csp_controller)
csp_dp = DeviceProxy(csp_server)
sdp_subarray_dp = DeviceProxy(sdp_subarray)
csp_subarray_leaf_node_dp = DeviceProxy(csp_subarray_leaf_node)
sdp_subarray_leaf_node_dp = DeviceProxy(sdp_subarray_leaf_node)
csp_master_leaf_node_dp = DeviceProxy(csp_master_leaf_node)

## 2 Load in Initial Parameters, Start Controller and Leaf Node

In [ ]:
sleep(3)

csp_controller_dp.adminMode = 0

sleep(6)

with open(
    INITIAL_PARAM,
    "r",
    encoding="utf-8",
) as json_data:
    d = json.load(json_data)
    csp_controller_dp.LoadDishCfg(json.dumps(d))

sleep(3)
print(csp_controller_dp.sourceDishVccConfig)
sleep(3)
csp_controller_dp.commandTimeout = 100

csp_controller_dp.cbfSimulationMode = False
sleep(3)
csp_master_leaf_node_dp.On()

# WAIT 100 Seconds

## 3 Assign Resources

### 3.1 CSP Assign Resources

In [ ]:
with open(CSP_ASSIGN_RESOURCES, "r", encoding="utf-8") as json_data:
    assign_resources_csp = json.load(json_data)
    assign_resources_csp["dish"]["receptor_ids"] = RECEPTORS
    print(assign_resources_csp)
    csp_subarray_leaf_node_dp.AssignResources(json.dumps(assign_resources_csp))

sleep(5)

print(csp_dp.obsState)

### 3.2 SDP Assign Resources

In [ ]:
sdp_subarray_leaf_node_dp.On()

sleep(3)

with open(TMC_ASSIGN_RESOURCES, "r", encoding="utf-8") as json_data:
    d = json.load(json_data)
    sdp_assign_resources = d["sdp"]
    sdp_assign_resources["resources"]["receptors"] = RECEPTORS
    sdp_subarray_leaf_node_dp.AssignResources(json.dumps(sdp_assign_resources))

sleep(10)
print(sdp_subarray_dp.obsState)

In [ ]:
receive_addresses = json.loads(sdp_subarray_dp.receiveAddresses)
print(receive_addresses)
print(receive_addresses["science"]["vis0"]["host"][0][1])

## 4 Configure Scans

### 4.1 CSP Configure Scan

In [ ]:
receive_addresses = json.loads(sdp_subarray_dp.receiveAddresses)

sleep(1)

with open(CONFIGURE_SCAN, "r", encoding="utf-8") as json_data:
    d = json.load(json_data)
    csp_configure_scan = d["csp"]
    csp_configure_scan["cbf"]["fsp"][0]["output_host"] = [[]]
    if SIMULATED_DELAY is True:
        csp_configure_scan["cbf"][
            "delay_model_subscription_point"
        ] = "ska_mid/tm_leaf_node/csp_subarray_01/delayModel"
    else:
        csp_configure_scan["cbf"][
            "delay_model_subscription_point"
        ] = "ska_mid/tm_leaf_node/csp_subarray01/delayModel"
    csp_configure_scan["cbf"]["fsp"][0]["output_host"][0] = [
        0,
        receive_addresses["science"]["vis0"]["host"][0][1],
    ]
    csp_configure_scan["cbf"]["fsp"][0]["output_port"] = [[0, 21000, 1]]
    print(csp_configure_scan)
    csp_subarray_leaf_node_dp.Configure(json.dumps(csp_configure_scan))

sleep(12)
print(csp_dp.obsState)

### 4.2 SIMULATE DELAY MODELS (OPTIONAL)

In [ ]:
ska_epoch_utc = Time(SKA_EPOCH, scale="utc")
ska_epoch_tai = ska_epoch_utc.unix_tai

start_utc_time = Time(datetime.now(timezone.utc) + timedelta(seconds=5), scale="utc")

target_epoch = start_utc_time.unix_tai - ska_epoch_tai

with open(DELAY_MODEL, encoding="utf-8") as f:
    delay_model = json.load(f)
delayModelProxy = DeviceProxy("ska_mid/tm_leaf_node/csp_subarray_01")
delay_model["start_validity_sec"] = target_epoch
print(json.dumps(delay_model))


tm_dp = DeviceProxy(tm_server)
tm_dp.delayModel = json.dumps(delay_model)

### 4.3 SDP Configure Scan

In [ ]:
with open(CONFIGURE_SCAN, "r", encoding="utf-8") as json_data:
    d = json.load(json_data)
    sdp_configure_Scan = d["sdp"]
    print(sdp_configure_Scan)
    sdp_subarray_leaf_node_dp.configure(json.dumps(sdp_configure_Scan))

sleep(3)
print(sdp_subarray_dp.obsState)

## 5 Start Scans

### 5.1 SDP Scan

In [ ]:
with open(SCAN_JSON, "r", encoding="utf-8") as json_data:
    d = json.load(json_data)
    sdp_scan = d["sdp"]
    print(sdp_scan)
    sdp_subarray_leaf_node_dp.scan(json.dumps(sdp_scan))

sleep(10)
print(sdp_subarray_dp.obsState)

### 5.2 CSP Scan

In [ ]:
with open(SCAN_JSON, "r", encoding="utf-8") as json_data:
    d = json.load(json_data)
    csp_scan = d["csp"]
    print(csp_scan)
    csp_subarray_leaf_node_dp.scan(json.dumps(csp_scan))

sleep(2)
print(csp_dp.obsState)

## 6 Cleanup

### 6.1 Cleanup CSP

In [ ]:
csp_subarray_leaf_node_dp.EndScan()
sleep(4)
csp_subarray_leaf_node_dp.End()
sleep(4)
csp_subarray_leaf_node_dp.ReleaseAllResources()

### 6.2 Cleanup SDP

In [ ]:
sdp_subarray_leaf_node_dp.EndScan()
# sdp_subarray_leaf_node_dp.End()
# sdp_subarray_leaf_node_dp.ReleaseAllResources()

### 6.3 Turn Off CSP

In [ ]:
csp_master_leaf_node_dp.Off()